In [1]:
import jupyter_black
jupyter_black.load()

import re
from collections import deque

In [2]:
with open("puzzle_input.txt") as f:
    text = f.read()

ranges_text, ingredients_text = text.split("\n\n")

ranges = [
    re.match(r"(?P<start>\d+)-(?P<end>\d+)", r).groupdict()
    for r in ranges_text.split("\n")
]

ingredients = ingredients_text.split("\n")

In [3]:
%%time


def check_range(query: str, start: str, end: str) -> bool:

    string_length = max(len(query), len(start), len(end))
    query_padded = query.zfill(string_length)
    start_padded = start.zfill(string_length)
    end_padded = end.zfill(string_length)

    for i in range(1, string_length + 1):
        if (query_padded[:i] < start_padded[:i]) or (query_padded[:i] > end_padded[:i]):
            return False

    return True


count = 0

for i in ingredients:
    for r in ranges:
        if check_range(i, r["start"], r["end"]):
            count += 1
            break

print(f"Puzzle 1 Answer: {count}")

Puzzle 1 Answer: 563
CPU times: user 66.8 ms, sys: 1.27 ms, total: 68.1 ms
Wall time: 67.3 ms


In [4]:
%%time

ranges_int = sorted(
    ((int(r["start"]), int(r["end"])) for r in ranges), key=lambda x: x[0]
)


def merge_ranges_int(ranges_int):

    ranges_int_queue = deque(ranges_int)

    merged_ranges_int = []

    i = 0
    remaining = len(ranges_int)
    start, end = ranges_int_queue.popleft()
    remaining -= 1

    while remaining > 0:
        next_start, next_end = ranges_int_queue.popleft()
        remaining -= 1
        if end < next_start:
            merged_ranges_int.append((start, end))
            start, end = next_start, next_end
        elif end < next_end:
            merged_ranges_int.append((start, next_end))
            start, end = ranges_int_queue.popleft()
            remaining -= 1

    if merged_ranges_int[-1] != (start, end):
        merged_ranges_int.append((start, end))

    return merged_ranges_int


m = ranges_int
l = len(ranges_int)
while True:
    m = merge_ranges_int(m)
    l_new = len(m)
    if l_new == l:
        break
    l = l_new

result = sum(j - i + 1 for (i, j) in m)

print(f"Puzzle 2 Answer: {result}")

Puzzle 2 Answer: 338693411431456
CPU times: user 178 μs, sys: 17 μs, total: 195 μs
Wall time: 192 μs
